# Polarsに関するTips

Polarsは、データフレーム操作を効率的に行うための強力なライブラリですが、さらに効果的に活用するためにはいくつかの便利なテクニックやコツを知っておくことが重要です。この章では、Polarsをよりスムーズに、かつ効率的に使用するための実践的なTipsを紹介します。これらのテクニックを使うことで、データの操作や処理を一層高速化し、作業の生産性を向上させることができます。

In [1]:
import panel as pn
pn.extension()
import polars as pl
from helper.jupyter import row

## メッセージチェン

Polarsでは、メソッドチェーンを活用することでデータ操作を効率的に行うことができますが、メソッドチェーンの途中でどのようにデータが変化するのかを把握するのは難しい場合があります。そこで、デバッグのために次の2つの方法を活用できます。

* `helper.polars.DataCapturer`: メソッドチェーンの任意の点でデータをキャプチャし、状態を確認できます。
* `helper.polars.PipeLogger`: メソッドチェーン全体の入出力と引数をキャプチャし、可視化します。

### DataCapturer

`DataCapturer`を使用すると、メソッドチェーン内の特定のポイントでデータの状態をキャプチャできます。以下は、`DataCapturer`の使い方を示す例です。

In [2]:
from helper.polars import DataCapturer

df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "score": [85, 90, 95, 80, 70],
    "department": ["HR", "IT", "HR", "IT", "HR"]
})

cap = DataCapturer() #❶

result = (
df
.filter(pl.col("age") > 30)  
.with_columns(
  (pl.col("score") * 1.1).alias("adjusted_score")
)
.pipe(cap.before_group) #❷
.group_by("department")
.agg([
  pl.col("adjusted_score").mean().alias("average_score"),
  pl.col("age").max().alias("max_age")
])
.pipe(cap.before_sort) #❷
.sort("average_score", descending=True)
)

row(cap.before_group, cap.before_sort) #❸

shape: (3, 5) name age score department adjusted_score str i64 i64 str f64 "Charlie" 35 95 "HR" 104.5 "David" 40 80 "IT" 88.0 "Eve" 45 70 "HR" 77.0 
 shape: (2, 3) department average_score max_age str f64 i64 "IT" 88.0 40 "HR" 90.75 45

このコードでは、`cap.before_group`でグループ化の前のデータ状態をキャプチャし、`cap.before_sort`でソート前のデータ状態をキャプチャしています。

❶メソッドチェーンを開始する前に、`DataCapturer()`のインスタンス`cap`を作成します。<br>
❷メソッドチェーンの特定の場所で、`.pipe(cap.name)`を挿入し、その時点のデータを`name`としてキャプチャします。<br>
❸キャプチャしたデータは`cap.name`でアクセスできます。<br>

### PipeLogger

`PipeLogger`を使用すると、メソッドチェーン内の各ステップで発生するデータの入出力を視覚的に確認できます。これにより、データがどのように変化しているかを逐一確認しながら処理を進めることができ、デバッグが容易になります。

以下のコード例では、`PipeLogger`を使用してデータフレームを操作し、処理中の入出力をJupyterLabで可視化する方法を示します。

In [3]:
from helper.polars import PipeLogger

result = (
PipeLogger(df) #❶
.filter(pl.col("age") > 30)  # 年齢が30歳以上の行をフィルタリング
.with_columns(
  (pl.col("score") * 1.1).alias("adjusted_score")  # スコアを調整
)
.group_by("department")  # 部署ごとにグループ化
.agg([
  pl.col("adjusted_score").mean().alias("average_score"),  # 各部署の平均スコアを計算
  pl.col("age").max().alias("max_age")  # 各部署の最年長者の年齢を取得
])
.sort("average_score", descending=True)  # 平均スコアで降順ソート
)

result #❷

`PipeLogger`を使うことで、メソッドチェーン内のデータの変化を視覚的に追跡し、デバッグがしやすくなります。

❶最初のオブジェクトである`df`を`PipeLogger()`でラップします。これにより、結果は`DataFrame`ではなく、すべての入出力をキャプチャする`PipeLogger`オブジェクトになります。`PipeLogger`は、メソッドチェーンの各ステップでデータの状態を保存します。

❷最後に、`PipeLogger`オブジェクトをセルの最後に配置し、インタラクティブなウィジェットとして結果を表示します。これにより、各メソッドの入出力を可視化することができます。